In [20]:
from google.colab import drive

drive.mount('/content/drive')
path = "/content/drive/My Drive/Datamining/TextMining/"
version = "v4"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp "/content/drive/My Drive/Datamining/TextMining/Notebooks/functions.py" .
import functions as f
import importlib
importlib.reload(f)
f.v()

2020-06-03 11:04


In [0]:
import pandas as pd
import csv
import numpy as np

In [4]:
df = f.loadTaggedDatasets(path + "Dataset/", version="v3")
df

,date,text,language_proba,month,is_repeated,is_spam,is_news,is_emoji,emoji_sent,emoji_sent_dico
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False,0.0,0.0,0.0
1,2019-05-10,Bitcoin FUDsters RN https://t.co/Yxo3TCxQci,0.449887,5,False,False,True,0.0,0.0,0.0
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False,0.0,0.0,0.0
3,2019-05-11,$1000 left!,0.247924,5,False,False,False,0.0,0.0,0.0
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
10329537,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11,False,True,True,0.0,0.0,0.0
10329538,2019-11-23,Bitcoin Suisse Certificates :) https://t.co/nd...,0.457228,11,False,False,True,0.0,0.0,0.0
10329539,2019-11-23,Register now for the early access of the Codug...,0.884063,11,False,True,False,0.0,0.0,0.0
10329540,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11,False,True,False,0.0,0.0,0.0


In [5]:
# Tiramos a la basura todo lo que tenga language_proba <= 0.5
df = df[df['language_proba'] > 0.5]
df

,date,text,language_proba,month,is_repeated,is_spam,is_news,is_emoji,emoji_sent,emoji_sent_dico
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False,0.0,0.0,0.0
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False,0.0,0.0,0.0
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False,0.0,0.0,0.0
5,2019-05-13,#retweet &amp; tag 🎁🎁We are glad to announce i...,0.855190,5,False,True,True,0.0,0.0,0.0
6,2019-05-13,"The last time Bitcoin broke $8,000: Microsoft...",0.978768,5,True,False,True,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
10329536,2019-11-23,Daily profit for HODLING BTC since 2013 Data t...,0.924064,11,False,False,False,0.0,0.0,0.0
10329537,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11,False,True,True,0.0,0.0,0.0
10329539,2019-11-23,Register now for the early access of the Codug...,0.884063,11,False,True,False,0.0,0.0,0.0
10329540,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11,False,True,False,0.0,0.0,0.0


In [6]:
df_sum = df.groupby('date').size().reset_index(name = 'total_count')
df_sum

,date,total_count
0,2019-05-10,32093
1,2019-05-11,57217
2,2019-05-12,48499
3,2019-05-13,70764
4,2019-05-14,67153
...,...,...
193,2019-11-19,38782
194,2019-11-20,38278
195,2019-11-21,43713
196,2019-11-22,46193


In [7]:
df_group = df.groupby(by=['date', 'is_repeated']).size().reset_index(name = 'count')
df_group

,date,is_repeated,count
0,2019-05-10,False,30107
1,2019-05-10,True,1986
2,2019-05-11,False,54374
3,2019-05-11,True,2843
4,2019-05-12,False,45789
...,...,...,...
391,2019-11-21,True,2538
392,2019-11-22,False,43732
393,2019-11-22,True,2461
394,2019-11-23,False,19872


In [8]:
df_sum = df_sum.merge(
    df_group[df_group['is_repeated'] == True].drop(['is_repeated'], axis=1).rename(columns={'count': 'repeated_count'}),
    left_on='date', right_on='date', how='left')
df_sum = df_sum.merge(
    df_group[df_group['is_repeated'] == False].drop(['is_repeated'], axis=1).rename(columns={'count': 'not_repeated_count'}),
    left_on='date', right_on='date', how='left')
df_sum

,date,total_count,repeated_count,not_repeated_count
0,2019-05-10,32093,1986,30107
1,2019-05-11,57217,2843,54374
2,2019-05-12,48499,2710,45789
3,2019-05-13,70764,3810,66954
4,2019-05-14,67153,3808,63345
...,...,...,...,...
193,2019-11-19,38782,2533,36249
194,2019-11-20,38278,2421,35857
195,2019-11-21,43713,2538,41175
196,2019-11-22,46193,2461,43732


In [9]:
# Para spam solo tenemos en cuenta los no repetidos
df = df[df['is_repeated'] == False]
df

,date,text,language_proba,month,is_repeated,is_spam,is_news,is_emoji,emoji_sent,emoji_sent_dico
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False,0.0,0.000,0.0
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False,0.0,0.000,0.0
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False,0.0,0.000,0.0
5,2019-05-13,#retweet &amp; tag 🎁🎁We are glad to announce i...,0.855190,5,False,True,True,0.0,0.000,0.0
9,2019-05-10,🚀🚀🚀 too the moon lol,0.726528,5,False,False,False,1.0,0.525,1.0
...,...,...,...,...,...,...,...,...,...,...
10329536,2019-11-23,Daily profit for HODLING BTC since 2013 Data t...,0.924064,11,False,False,False,0.0,0.000,0.0
10329537,2019-11-23,Happy #FibonacciDay A while back I created a...,0.939878,11,False,True,True,0.0,0.000,0.0
10329539,2019-11-23,Register now for the early access of the Codug...,0.884063,11,False,True,False,0.0,0.000,0.0
10329540,2019-11-23,@btc @btc Do you know that BTC Baskets isn't...,0.925460,11,False,True,False,0.0,0.000,0.0


In [10]:
df_group = df.groupby(by=['date', 'is_spam']).size().reset_index(name = 'count')
df_group

,date,is_spam,count
0,2019-05-10,False,20079
1,2019-05-10,True,10028
2,2019-05-11,False,37087
3,2019-05-11,True,17287
4,2019-05-12,False,29285
...,...,...,...
391,2019-11-21,True,15169
392,2019-11-22,False,29254
393,2019-11-22,True,14478
394,2019-11-23,False,12775


In [11]:
df_sum = df_sum.merge(
    df_group[df_group['is_spam'] == True].drop(['is_spam'], axis=1).rename(columns={'count': 'spam_count'}),
    left_on='date', right_on='date', how='left')
df_sum = df_sum.merge(
    df_group[df_group['is_spam'] == False].drop(['is_spam'], axis=1).rename(columns={'count': 'not_spam_count'}),
    left_on='date', right_on='date', how='left')
df_sum

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count
0,2019-05-10,32093,1986,30107,10028,20079
1,2019-05-11,57217,2843,54374,17287,37087
2,2019-05-12,48499,2710,45789,16504,29285
3,2019-05-13,70764,3810,66954,21559,45395
4,2019-05-14,67153,3808,63345,22211,41134
...,...,...,...,...,...,...
193,2019-11-19,38782,2533,36249,14480,21769
194,2019-11-20,38278,2421,35857,14567,21290
195,2019-11-21,43713,2538,41175,15169,26006
196,2019-11-22,46193,2461,43732,14478,29254


In [12]:
# Para noticias solo tenemos en cuenta los no spam
df = df[df['is_spam'] == False]
df

,date,text,language_proba,month,is_repeated,is_spam,is_news,is_emoji,emoji_sent,emoji_sent_dico
0,2019-05-10,@mark_dow Bitcoin is the one imv,0.980272,5,False,False,False,0.0,0.000,0.0
2,2019-05-10,"@KennETHbosak ""Banking on Bitcoin"" on Netflix.",0.685419,5,False,False,False,0.0,0.000,0.0
4,2019-05-11,Never too late to buy. The best timing to buy ...,0.937032,5,False,False,False,0.0,0.000,0.0
9,2019-05-10,🚀🚀🚀 too the moon lol,0.726528,5,False,False,False,1.0,0.525,1.0
10,2019-05-10,"$BTC all targets hit, more than 90% profit 💰",0.637279,5,False,False,False,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...
10329531,2019-11-23,Bitcoin- just doing the SpaceX rocket factory,0.875814,11,False,False,False,0.0,0.000,0.0
10329532,2019-11-23,We are super happy to announce that TZC is add...,0.904834,11,False,False,False,0.0,0.000,0.0
10329533,2019-11-23,@EarlofTyrone @wenlembo @coindesk @La__Cuen @H...,0.972412,11,False,False,False,0.0,0.000,0.0
10329536,2019-11-23,Daily profit for HODLING BTC since 2013 Data t...,0.924064,11,False,False,False,0.0,0.000,0.0


In [13]:
df_group = df.groupby(by=['date', 'is_news']).size().reset_index(name = 'count')
df_group

,date,is_news,count
0,2019-05-10,False,15101
1,2019-05-10,True,4978
2,2019-05-11,False,27907
3,2019-05-11,True,9180
4,2019-05-12,False,20704
...,...,...,...
391,2019-11-21,True,6913
392,2019-11-22,False,20887
393,2019-11-22,True,8367
394,2019-11-23,False,8631


In [14]:
df_sum = df_sum.merge(
    df_group[df_group['is_news'] == True].drop(['is_news'], axis=1).rename(columns={'count': 'news_count'}),
    left_on='date', right_on='date', how='left')
df_sum = df_sum.merge(
    df_group[df_group['is_news'] == False].drop(['is_news'], axis=1).rename(columns={'count': 'not_news_count'}),
    left_on='date', right_on='date', how='left')
df_sum

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count,news_count,not_news_count
0,2019-05-10,32093,1986,30107,10028,20079,4978,15101
1,2019-05-11,57217,2843,54374,17287,37087,9180,27907
2,2019-05-12,48499,2710,45789,16504,29285,8581,20704
3,2019-05-13,70764,3810,66954,21559,45395,12009,33386
4,2019-05-14,67153,3808,63345,22211,41134,11169,29965
...,...,...,...,...,...,...,...,...
193,2019-11-19,38782,2533,36249,14480,21769,6292,15477
194,2019-11-20,38278,2421,35857,14567,21290,6352,14938
195,2019-11-21,43713,2538,41175,15169,26006,6913,19093
196,2019-11-22,46193,2461,43732,14478,29254,8367,20887


In [15]:
# En este punto df ya tiene solamente los que no son spam, que es lo mismo que necesitamos para los emojis
df_group = df.groupby(by=['date', 'emoji_sent_dico']).size().reset_index(name = 'count')
df_group

,date,emoji_sent_dico,count
0,2019-05-10,-1.0,8
1,2019-05-10,0.0,19856
2,2019-05-10,1.0,168
3,2019-05-11,-1.0,5
4,2019-05-11,0.0,36734
...,...,...,...
584,2019-11-22,0.0,27634
585,2019-11-22,1.0,1491
586,2019-11-23,-1.0,8
587,2019-11-23,0.0,11958


In [16]:
df_sum = df_sum.merge(
    df_group[df_group['emoji_sent_dico'] == '1.0'].drop(['emoji_sent_dico'], axis=1).rename(columns={'count': 'emoji_positive_count'}),
    left_on='date', right_on='date', how='left')
df_sum = df_sum.merge(
    df_group[df_group['emoji_sent_dico'] == '-1.0'].drop(['emoji_sent_dico'], axis=1).rename(columns={'count': 'emoji_negative_count'}),
    left_on='date', right_on='date', how='left')
df_sum['emoji_negative_count'] = df_sum['emoji_negative_count'].apply(lambda x: 0.0 if np.isnan(x) else x)
df_sum

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count,news_count,not_news_count,emoji_positive_count,emoji_negative_count
0,2019-05-10,32093,1986,30107,10028,20079,4978,15101,168,8.0
1,2019-05-11,57217,2843,54374,17287,37087,9180,27907,284,5.0
2,2019-05-12,48499,2710,45789,16504,29285,8581,20704,246,3.0
3,2019-05-13,70764,3810,66954,21559,45395,12009,33386,305,11.0
4,2019-05-14,67153,3808,63345,22211,41134,11169,29965,291,1.0
...,...,...,...,...,...,...,...,...,...,...
193,2019-11-19,38782,2533,36249,14480,21769,6292,15477,722,7.0
194,2019-11-20,38278,2421,35857,14567,21290,6352,14938,867,9.0
195,2019-11-21,43713,2538,41175,15169,26006,6913,19093,996,8.0
196,2019-11-22,46193,2461,43732,14478,29254,8367,20887,1491,16.0


In [17]:
df_sum['total_count_normalized'] = df_sum['total_count'] / df_sum['total_count'].max() # Se puede usar tranqui porque el max está en Junio
df_sum['repeated_ratio'] = df_sum['repeated_count'] / df_sum['total_count']
df_sum['not_repeated_ratio'] = df_sum['not_repeated_count'] / df_sum['total_count']
df_sum['spam_ratio'] = df_sum['spam_count'] / df_sum['not_repeated_count']
df_sum['not_spam_ratio'] = df_sum['not_spam_count'] / df_sum['not_repeated_count']
df_sum['news_ratio'] = df_sum['news_count'] / df_sum['not_spam_count']
df_sum['not_news_ratio'] = df_sum['not_news_count'] / df_sum['not_spam_count']
df_sum['emoji_positive_ratio'] = df_sum['emoji_positive_count'] / df_sum['not_spam_count']
df_sum['emoji_negative_ratio'] = df_sum['emoji_negative_count'] / df_sum['not_spam_count']
df_sum

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count,news_count,not_news_count,emoji_positive_count,emoji_negative_count,total_count_normalized,repeated_ratio,not_repeated_ratio,spam_ratio,not_spam_ratio,news_ratio,not_news_ratio,emoji_positive_ratio,emoji_negative_ratio
0,2019-05-10,32093,1986,30107,10028,20079,4978,15101,168,8.0,0.193839,0.061883,0.938117,0.333079,0.666921,0.247921,0.752079,0.008367,0.000398
1,2019-05-11,57217,2843,54374,17287,37087,9180,27907,284,5.0,0.345586,0.049688,0.950312,0.317928,0.682072,0.247526,0.752474,0.007658,0.000135
2,2019-05-12,48499,2710,45789,16504,29285,8581,20704,246,3.0,0.292930,0.055877,0.944123,0.360436,0.639564,0.293017,0.706983,0.008400,0.000102
3,2019-05-13,70764,3810,66954,21559,45395,12009,33386,305,11.0,0.427409,0.053841,0.946159,0.321997,0.678003,0.264545,0.735455,0.006719,0.000242
4,2019-05-14,67153,3808,63345,22211,41134,11169,29965,291,1.0,0.405599,0.056706,0.943294,0.350635,0.649365,0.271527,0.728473,0.007074,0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2019-11-19,38782,2533,36249,14480,21769,6292,15477,722,7.0,0.234240,0.065314,0.934686,0.399459,0.600541,0.289035,0.710965,0.033166,0.000322
194,2019-11-20,38278,2421,35857,14567,21290,6352,14938,867,9.0,0.231196,0.063248,0.936752,0.406253,0.593747,0.298356,0.701644,0.040723,0.000423
195,2019-11-21,43713,2538,41175,15169,26006,6913,19093,996,8.0,0.264023,0.058061,0.941939,0.368403,0.631597,0.265823,0.734177,0.038299,0.000308
196,2019-11-22,46193,2461,43732,14478,29254,8367,20887,1491,16.0,0.279002,0.053276,0.946724,0.331062,0.668938,0.286012,0.713988,0.050967,0.000547


In [29]:
df_lr = pd.read_csv(f'{path}Dataset/{version}/RegresionLinear.csv')
df_lr

,Fecha,IndiceTexto
0,2019-05-10,1.503588
1,2019-05-11,2.675903
2,2019-05-12,2.870511
3,2019-05-13,3.126731
4,2019-05-14,2.699624
...,...,...
193,2019-11-19,0.758773
194,2019-11-20,0.471242
195,2019-11-21,0.765482
196,2019-11-22,1.456646


In [31]:
df_lr['IndiceTexto'] = df_lr['IndiceTexto'].astype(float)
df_lr = df_lr.rename(columns={'Fecha': 'date', 'IndiceTexto': 'linear_regression_index'})
df_lr

,date,linear_regression_index
0,2019-05-10,1.503588
1,2019-05-11,2.675903
2,2019-05-12,2.870511
3,2019-05-13,3.126731
4,2019-05-14,2.699624
...,...,...
193,2019-11-19,0.758773
194,2019-11-20,0.471242
195,2019-11-21,0.765482
196,2019-11-22,1.456646


In [32]:
df_sum = df_sum.merge(df_lr, left_on='date', right_on='date', how='left')
df_sum

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count,news_count,not_news_count,emoji_positive_count,emoji_negative_count,total_count_normalized,repeated_ratio,not_repeated_ratio,spam_ratio,not_spam_ratio,news_ratio,not_news_ratio,emoji_positive_ratio,emoji_negative_ratio,linear_regression_index
0,2019-05-10,32093,1986,30107,10028,20079,4978,15101,168,8.0,0.193839,0.061883,0.938117,0.333079,0.666921,0.247921,0.752079,0.008367,0.000398,1.503588
1,2019-05-11,57217,2843,54374,17287,37087,9180,27907,284,5.0,0.345586,0.049688,0.950312,0.317928,0.682072,0.247526,0.752474,0.007658,0.000135,2.675903
2,2019-05-12,48499,2710,45789,16504,29285,8581,20704,246,3.0,0.292930,0.055877,0.944123,0.360436,0.639564,0.293017,0.706983,0.008400,0.000102,2.870511
3,2019-05-13,70764,3810,66954,21559,45395,12009,33386,305,11.0,0.427409,0.053841,0.946159,0.321997,0.678003,0.264545,0.735455,0.006719,0.000242,3.126731
4,2019-05-14,67153,3808,63345,22211,41134,11169,29965,291,1.0,0.405599,0.056706,0.943294,0.350635,0.649365,0.271527,0.728473,0.007074,0.000024,2.699624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2019-11-19,38782,2533,36249,14480,21769,6292,15477,722,7.0,0.234240,0.065314,0.934686,0.399459,0.600541,0.289035,0.710965,0.033166,0.000322,0.758773
194,2019-11-20,38278,2421,35857,14567,21290,6352,14938,867,9.0,0.231196,0.063248,0.936752,0.406253,0.593747,0.298356,0.701644,0.040723,0.000423,0.471242
195,2019-11-21,43713,2538,41175,15169,26006,6913,19093,996,8.0,0.264023,0.058061,0.941939,0.368403,0.631597,0.265823,0.734177,0.038299,0.000308,0.765482
196,2019-11-22,46193,2461,43732,14478,29254,8367,20887,1491,16.0,0.279002,0.053276,0.946724,0.331062,0.668938,0.286012,0.713988,0.050967,0.000547,1.456646


In [0]:
df_sum.to_csv(f'{path}Dataset/{version}/tagged_summary.csv.gz',
                    index = False, compression='gzip', escapechar='\\', quotechar='"', encoding = "utf-8",
                    quoting=csv.QUOTE_NONNUMERIC, sep=";")

In [0]:
df_sum.total_count.sum()

8829168

Reviso a ver como quedaron guardados los dataframes

In [34]:
dfv4 = pd.read_csv(f'{path}Dataset/{version}/tagged_summary.csv.gz',
                    compression='gzip', escapechar='\\', quotechar='"', encoding = "utf-8",
                    quoting=csv.QUOTE_NONNUMERIC, sep=";")
dfv4

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count,news_count,not_news_count,emoji_positive_count,emoji_negative_count,total_count_normalized,repeated_ratio,not_repeated_ratio,spam_ratio,not_spam_ratio,news_ratio,not_news_ratio,emoji_positive_ratio,emoji_negative_ratio,linear_regression_index
0,2019-05-10,32093.0,1986.0,30107.0,10028.0,20079.0,4978.0,15101.0,168.0,8.0,0.193839,0.061883,0.938117,0.333079,0.666921,0.247921,0.752079,0.008367,0.000398,1.503588
1,2019-05-11,57217.0,2843.0,54374.0,17287.0,37087.0,9180.0,27907.0,284.0,5.0,0.345586,0.049688,0.950312,0.317928,0.682072,0.247526,0.752474,0.007658,0.000135,2.675903
2,2019-05-12,48499.0,2710.0,45789.0,16504.0,29285.0,8581.0,20704.0,246.0,3.0,0.292930,0.055877,0.944123,0.360436,0.639564,0.293017,0.706983,0.008400,0.000102,2.870511
3,2019-05-13,70764.0,3810.0,66954.0,21559.0,45395.0,12009.0,33386.0,305.0,11.0,0.427409,0.053841,0.946159,0.321997,0.678003,0.264545,0.735455,0.006719,0.000242,3.126731
4,2019-05-14,67153.0,3808.0,63345.0,22211.0,41134.0,11169.0,29965.0,291.0,1.0,0.405599,0.056706,0.943294,0.350635,0.649365,0.271527,0.728473,0.007074,0.000024,2.699624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2019-11-19,38782.0,2533.0,36249.0,14480.0,21769.0,6292.0,15477.0,722.0,7.0,0.234240,0.065314,0.934686,0.399459,0.600541,0.289035,0.710965,0.033166,0.000322,0.758773
194,2019-11-20,38278.0,2421.0,35857.0,14567.0,21290.0,6352.0,14938.0,867.0,9.0,0.231196,0.063248,0.936752,0.406253,0.593747,0.298356,0.701644,0.040723,0.000423,0.471242
195,2019-11-21,43713.0,2538.0,41175.0,15169.0,26006.0,6913.0,19093.0,996.0,8.0,0.264023,0.058061,0.941939,0.368403,0.631597,0.265823,0.734177,0.038299,0.000308,0.765482
196,2019-11-22,46193.0,2461.0,43732.0,14478.0,29254.0,8367.0,20887.0,1491.0,16.0,0.279002,0.053276,0.946724,0.331062,0.668938,0.286012,0.713988,0.050967,0.000547,1.456646


In [0]:
dfv1 = pd.read_csv(f'{path}Dataset/v1/tagged_summary.csv.gz',
                    compression='gzip', escapechar='\\', quotechar='"', encoding = "utf-8",
                    quoting=csv.QUOTE_NONNUMERIC, sep=";")
dfv1

,date,total_count,repeated_count,not_repeated_count,spam_count,not_spam_count,news_count,not_news_count,repeated_ratio,not_repeated_ratio,spam_ratio,not_spam_ratio,news_ratio,not_news_ratio
0,2019-05-10,32093.0,1986.0,30107.0,10028.0,20079.0,4978.0,15101.0,0.061883,0.938117,0.333079,0.666921,0.247921,0.752079
1,2019-05-11,57226.0,2843.0,54383.0,17290.0,37093.0,9180.0,27913.0,0.049680,0.950320,0.317930,0.682070,0.247486,0.752514
2,2019-05-12,48499.0,2710.0,45789.0,16504.0,29285.0,8581.0,20704.0,0.055877,0.944123,0.360436,0.639564,0.293017,0.706983
3,2019-05-13,70764.0,3810.0,66954.0,21559.0,45395.0,12009.0,33386.0,0.053841,0.946159,0.321997,0.678003,0.264545,0.735455
4,2019-05-14,67155.0,3808.0,63347.0,22211.0,41136.0,11169.0,29967.0,0.056705,0.943295,0.350624,0.649376,0.271514,0.728486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,2019-11-19,38783.0,2533.0,36250.0,14480.0,21770.0,6292.0,15478.0,0.065312,0.934688,0.399448,0.600552,0.289022,0.710978
194,2019-11-20,38278.0,2421.0,35857.0,14567.0,21290.0,6352.0,14938.0,0.063248,0.936752,0.406253,0.593747,0.298356,0.701644
195,2019-11-21,43713.0,2538.0,41175.0,15169.0,26006.0,6913.0,19093.0,0.058061,0.941939,0.368403,0.631597,0.265823,0.734177
196,2019-11-22,46193.0,2461.0,43732.0,14478.0,29254.0,8367.0,20887.0,0.053276,0.946724,0.331062,0.668938,0.286012,0.713988
